In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

In [2]:
def calculate_signal(
    df,
    col,
    target_col,
    exposure_col=None,
):

    # aggregate
    _ = df.groupby(col, dropna=False, observed=0)
    if exposure_col is not None:
        _ = _.agg(
            count=(target_col, np.size),
            exposure=(exposure_col, "sum"),
            signal=(target_col, "sum"),
        )
        _.rename(columns={"signal": target_col, "exposure": exposure_col}, inplace=True)
    else:
        _ = _.agg(
            count=(target_col, np.size),
            signal=(target_col, "sum"),
        )
        _.rename(
            columns={
                "signal": target_col,
            },
            inplace=True,
        )

    # count based pct & rates
    _["count_pct"] = _["count"].div(_["count"].sum()).mul(100)
    _[f"{target_col}_pct"] = _[f"{target_col}"].div(_[f"{target_col}"].sum()).mul(100)
    _[f"{target_col}_rate"] = _[f"{target_col}"].div(_["count"]).mul(100)

    # exposure based pct & rates
    if exposure_col is not None:
        _[f"{exposure_col}_pct"] = _[exposure_col].div(_[exposure_col].sum()).mul(100)
        _[f"{target_col}_{exposure_col}_rate"] = (
            _[f"{target_col}"].div(_[exposure_col]).mul(100)
        )

    return _

### calcs

In [3]:
df = pd.read_csv("../toydata/titanic.csv")
print(f"{df.shape}")
df.head(1)

(891, 15)


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False


In [4]:
edges = [0, 20, 40, 100]
df["age_binned"] = pd.cut(df["age"], bins=edges, right=False)
edges = [0, 10, 50, 100, 600]
df["fare_binned"] = pd.cut(df["fare"], bins=edges, right=False)

df["age_fare_binned"] = (
    df["age_binned"].astype(str) + "_" + df["fare_binned"].astype(str)
)

_ = calculate_signal(
    df, target_col="survived", col="age_fare_binned", exposure_col="fare"
)
_["age_binned"] = [i for i, j in _.index.str.split("_")]
_["fare_binned"] = [j for i, j in _.index.str.split("_")]

In [5]:
_.head(5)

,count,fare,survived,count_pct,survived_pct,survived_rate,fare_pct,survived_fare_rate,age_binned,fare_binned
age_fare_binned,,,,,,,,,,
"[0, 20)_[0, 10)",44,340.5668,15,4.938272,4.385965,34.090909,1.186894,4.404422,"[0, 20)","[0, 10)"
"[0, 20)_[10, 50)",101,2457.0833,50,11.335578,14.619883,49.504950,8.563071,2.034933,"[0, 20)","[10, 50)"
"[0, 20)_[100, 600)",11,1836.0208,8,1.234568,2.339181,72.727273,6.398634,0.435725,"[0, 20)","[100, 600)"
"[0, 20)_[50, 100)",8,580.6667,6,0.897868,1.754386,75.000000,2.023656,1.033295,"[0, 20)","[50, 100)"
"[20, 40)_[0, 10)",158,1221.4286,29,17.732884,8.479532,18.354430,4.256746,2.374269,"[20, 40)","[0, 10)"


In [13]:
_.pivot(index="age_binned", columns="fare_binned", values="count").round(2).style.map(
    lambda v: "background-color:darkred;" if v < _["count"].sum() / len(_) else None
)

fare_binned,"[0, 10)","[10, 50)","[100, 600)","[50, 100)"
age_binned,,,,
"[0, 20)",44,101,11,8
"[20, 40)",158,157,23,49
"[40, 100)",34,82,14,33
nan,100,54,5,18


In [15]:
_2 = _.pivot(index="age_binned", columns="fare_binned", values="survived_rate").round(2)
_2.style.background_gradient(cmap="RdYlGn")

fare_binned,"[0, 10)","[10, 50)","[100, 600)","[50, 100)"
age_binned,,,,
"[0, 20)",34.090000,49.500000,72.730000,75.000000
"[20, 40)",18.350000,42.680000,78.260000,73.470000
"[40, 100)",8.820000,34.150000,71.430000,60.610000
nan,20.000000,38.890000,60.000000,44.440000


In [ ]:
_.pivot(index="age_binned", columns="fare_binned", values="fare").round(2).style.map(
    lambda v: "background-color:darkred;" if v < _["fare"].sum() / len(_) else None
)

fare_binned,"[0, 10)","[10, 50)","[100, 600)","[50, 100)"
age_binned,,,,
"[0, 20)",340.570000,2457.080000,1836.020000,580.670000
"[20, 40)",1221.430000,3161.010000,4935.430000,3426.500000
"[40, 100)",249.670000,1971.650000,2300.930000,2290.930000
nan,713.350000,1200.680000,840.360000,1167.670000


In [ ]:
_2 = _.pivot(
    index="age_binned", columns="fare_binned", values="survived_fare_rate"
).round(2)
_2.style.background_gradient(cmap="RdYlGn")

fare_binned,"[0, 10)","[10, 50)","[100, 600)","[50, 100)"
age_binned,,,,
"[0, 20)",4.400000,2.030000,0.440000,1.030000
"[20, 40)",2.370000,2.120000,0.360000,1.050000
"[40, 100)",1.200000,1.420000,0.430000,0.870000
nan,2.800000,1.750000,0.360000,0.690000
